# EventReport API Documentation

This notebook provides interactive documentation for all API endpoints in the EventReport backend.
You can execute each cell to test the API endpoints directly.

## Setup

First, let's set up the base configuration and helper functions.

In [ ]:
import requests
import json
from datetime import datetime
from IPython.display import display, HTML, JSON

BASE_URL = "http://backend:8000"

ACCESS_TOKEN = None

def get_headers():
    headers = {"Content-Type": "application/json"}
    if ACCESS_TOKEN:
        headers["Authorization"] = f"Bearer {ACCESS_TOKEN}"
    return headers

def pretty_print(response):
    print(f"Status Code: {response.status_code}")
    print(f"URL: {response.url}")
    print("-" * 50)
    try:
        data = response.json()
        print(json.dumps(data, indent=2, default=str))
        return data
    except:
        print(response.text)
        return response.text

print("Setup complete! BASE_URL:", BASE_URL)

---
## Health Check

### `GET /` - Check if API is running

In [ ]:
response = requests.get(f"{BASE_URL}/")
pretty_print(response)

---
# Authentication Endpoints

## `POST /auth/register` - Register a new user

**Request Body:**
```json
{
    "first_name": "string",
    "last_name": "string",
    "email": "string (valid email)",
    "password": "string (min 8 chars, 1 uppercase, 1 digit, 1 special char)",
    "phone": "string"
}
```

**Response:** `201 Created`

In [ ]:
register_data = {
    "first_name": "John",
    "last_name": "Doe",
    "email": "john.doe@example.com",
    "password": "SecurePass123!",
    "phone": "+1234567890"
}

response = requests.post(
    f"{BASE_URL}/auth/register",
    json=register_data
)
pretty_print(response)

## `POST /auth/login` - Login user

**Request Body:**
```json
{
    "email": "string",
    "password": "string"
}
```

**Response:** Returns JWT access token

In [ ]:
login_data = {
    "email": "john.doe@example.com",
    "password": "SecurePass123!"
}

response = requests.post(
    f"{BASE_URL}/auth/login",
    json=login_data
)
result = pretty_print(response)

if response.status_code == 200:
    ACCESS_TOKEN = result["access_token"]
    print("\n" + "="*50)
    print("Token stored! You can now make authenticated requests.")

## `GET /auth/me` - Get current user info

**Headers:** `Authorization: Bearer <token>`

**Response:** User details

In [ ]:
response = requests.get(
    f"{BASE_URL}/auth/me",
    headers=get_headers()
)
pretty_print(response)

## `POST /auth/forgot-password` - Request password reset

**Request Body:**
```json
{
    "email": "string"
}
```

In [ ]:
forgot_data = {
    "email": "john.doe@example.com"
}

response = requests.post(
    f"{BASE_URL}/auth/forgot-password",
    json=forgot_data
)
pretty_print(response)

## `POST /auth/reset-password` - Reset password with code

**Request Body:**
```json
{
    "email": "string",
    "code": "string (6-digit code from email)",
    "new_password": "string"
}
```

In [ ]:
reset_data = {
    "email": "john.doe@example.com",
    "code": "123456",
    "new_password": "NewSecurePass123!"
}

response = requests.post(
    f"{BASE_URL}/auth/reset-password",
    json=reset_data
)
pretty_print(response)

---
# Events Endpoints

## `POST /events` - Create a new event

**Headers:** `Authorization: Bearer <token>`

**Request Body:**
```json
{
    "location": {
        "type": "Point",
        "coordinates": [longitude, latitude],
        "address": "optional string"
    },
    "alert_code": "GREEN | YELLOW | ORANGE | RED",
    "description": "string",
    "tags": ["tag1", "tag2"]
}
```

In [ ]:
event_data = {
    "location": {
        "type": "Point",
        "coordinates": [-98.5795, 39.8283],
        "address": "Kansas, USA"
    },
    "alert_code": "YELLOW",
    "description": "Test event from Jupyter notebook - Weather warning",
    "tags": ["weather", "test", "jupyter"]
}

response = requests.post(
    f"{BASE_URL}/events",
    json=event_data,
    headers=get_headers()
)
result = pretty_print(response)

if response.status_code == 201:
    CREATED_EVENT_ID = result["id"]
    print(f"\nCreated event ID: {CREATED_EVENT_ID}")

## `GET /events` - Get all events

**Query Parameters:**
- `alert_code` (optional): Filter by alert code (GREEN, YELLOW, ORANGE, RED)
- `tags` (optional): Filter by tags (comma-separated)
- `skip` (default: 0): Number of records to skip
- `limit` (default: 50, max: 100): Number of records to return

In [ ]:
params = {
    "skip": 0,
    "limit": 10
}

response = requests.get(
    f"{BASE_URL}/events",
    params=params
)
result = pretty_print(response)
print(f"\nTotal events returned: {len(result) if isinstance(result, list) else 0}")

## `GET /events/geojson` - Get events as GeoJSON

Returns events in GeoJSON FeatureCollection format for mapping.

**Query Parameters:**
- `alert_code` (optional): Filter by alert code
- `tags` (optional): Filter by tags
- `limit` (default: 100, max: 500): Number of records

In [ ]:
params = {
    "limit": 5
}

response = requests.get(
    f"{BASE_URL}/events/geojson",
    params=params
)
result = pretty_print(response)
print(f"\nFeatures count: {len(result.get('features', [])) if isinstance(result, dict) else 0}")

## `GET /events/nearby` - Get events near a location

**Query Parameters (all required):**
- `longitude`: Longitude (-180 to 180)
- `latitude`: Latitude (-90 to 90)
- `max_distance` (default: 5000): Max distance in meters (100-50000)
- `limit` (default: 20, max: 100): Number of records

In [ ]:
params = {
    "longitude": -98.5795,
    "latitude": 39.8283,
    "max_distance": 500000,
    "limit": 10
}

response = requests.get(
    f"{BASE_URL}/events/nearby",
    params=params
)
result = pretty_print(response)
print(f"\nNearby events found: {len(result) if isinstance(result, list) else 0}")

## `GET /events/{event_id}` - Get a specific event

In [ ]:
events_response = requests.get(f"{BASE_URL}/events", params={"limit": 1})
events = events_response.json()

if events:
    event_id = events[0]["id"]
    print(f"Fetching event with ID: {event_id}\n")
    
    response = requests.get(f"{BASE_URL}/events/{event_id}")
    pretty_print(response)
else:
    print("No events found in database")

## `PUT /events/{event_id}` - Update an event

**Headers:** `Authorization: Bearer <token>`

**Request Body (all fields optional):**
```json
{
    "location": {...},
    "alert_code": "GREEN | YELLOW | ORANGE | RED",
    "description": "string",
    "tags": ["tag1", "tag2"]
}
```

In [ ]:
events_response = requests.get(f"{BASE_URL}/events", params={"limit": 1})
events = events_response.json()

if events:
    event_id = events[0]["id"]
    print(f"Updating event with ID: {event_id}\n")
    
    update_data = {
        "description": "Updated description from Jupyter notebook",
        "alert_code": "ORANGE",
        "tags": ["updated", "jupyter"]
    }
    
    response = requests.put(
        f"{BASE_URL}/events/{event_id}",
        json=update_data,
        headers=get_headers()
    )
    pretty_print(response)
else:
    print("No events found to update")

## `DELETE /events/{event_id}` - Delete an event

**Headers:** `Authorization: Bearer <token>`

**Response:** `204 No Content`

In [ ]:
event_id_to_delete = "REPLACE_WITH_EVENT_ID"

if event_id_to_delete != "REPLACE_WITH_EVENT_ID":
    response = requests.delete(
        f"{BASE_URL}/events/{event_id_to_delete}",
        headers=get_headers()
    )
    print(f"Status Code: {response.status_code}")
    if response.status_code == 204:
        print("Event deleted successfully!")
    else:
        pretty_print(response)
else:
    print("Please replace 'REPLACE_WITH_EVENT_ID' with an actual event ID to delete")

---
# Image Endpoints

## `POST /events/{event_id}/image` - Upload image to event

**Headers:** `Authorization: Bearer <token>`

**Content-Type:** `multipart/form-data`

**Allowed file types:** JPEG, PNG, GIF, WEBP

In [ ]:
event_id = "REPLACE_WITH_EVENT_ID"
image_path = "/path/to/image.jpg"

if event_id != "REPLACE_WITH_EVENT_ID" and image_path != "/path/to/image.jpg":
    with open(image_path, "rb") as f:
        files = {"file": ("image.jpg", f, "image/jpeg")}
        headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"} if ACCESS_TOKEN else {}
        
        response = requests.post(
            f"{BASE_URL}/events/{event_id}/image",
            files=files,
            headers=headers
        )
        pretty_print(response)
else:
    print("Please set event_id and image_path variables to upload an image")

## `GET /events/{event_id}/image` - Get event image

Returns the image file as a stream.

In [ ]:
from IPython.display import Image, display

events_response = requests.get(f"{BASE_URL}/events", params={"limit": 50})
events = events_response.json()

event_with_image = None
for event in events:
    if event.get("image_id"):
        event_with_image = event
        break

if event_with_image:
    event_id = event_with_image["id"]
    print(f"Fetching image for event: {event_id}\n")
    
    response = requests.get(f"{BASE_URL}/events/{event_id}/image")
    if response.status_code == 200:
        display(Image(response.content))
    else:
        print(f"Error: {response.status_code}")
else:
    print("No events with images found")

## `GET /images/{image_id}` - Get image by ID

In [ ]:
from IPython.display import Image, display

image_id = "REPLACE_WITH_IMAGE_ID"

if image_id != "REPLACE_WITH_IMAGE_ID":
    response = requests.get(f"{BASE_URL}/images/{image_id}")
    if response.status_code == 200:
        display(Image(response.content))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
else:
    print("Please set image_id variable")

---
# Admin Endpoints

## `POST /admins` - Create admin notification contact

**Headers:** `Authorization: Bearer <token>`

**Request Body:**
```json
{
    "first_name": "string",
    "last_name": "string",
    "email": "string",
    "phone": "string"
}
```

In [ ]:
admin_data = {
    "first_name": "Admin",
    "last_name": "User",
    "email": "admin@example.com",
    "phone": "+1234567890"
}

response = requests.post(
    f"{BASE_URL}/admins",
    json=admin_data,
    headers=get_headers()
)
pretty_print(response)

## `GET /admins` - Get all admin contacts

**Headers:** `Authorization: Bearer <token>`

In [ ]:
response = requests.get(
    f"{BASE_URL}/admins",
    headers=get_headers()
)
result = pretty_print(response)
print(f"\nTotal admins: {len(result) if isinstance(result, list) else 0}")

## `DELETE /admins/{admin_id}` - Delete admin contact

**Headers:** `Authorization: Bearer <token>`

**Response:** `204 No Content`

In [ ]:
admin_id_to_delete = "REPLACE_WITH_ADMIN_ID"

if admin_id_to_delete != "REPLACE_WITH_ADMIN_ID":
    response = requests.delete(
        f"{BASE_URL}/admins/{admin_id_to_delete}",
        headers=get_headers()
    )
    print(f"Status Code: {response.status_code}")
    if response.status_code == 204:
        print("Admin deleted successfully!")
    else:
        pretty_print(response)
else:
    print("Please replace 'REPLACE_WITH_ADMIN_ID' with an actual admin ID")

---
# SMS Testing

## `POST /sms/test` - Send test SMS

**Headers:** `Authorization: Bearer <token>`

**Query Parameters:**
- `phone`: Phone number with country code (e.g., +1234567890)
- `message` (optional): Custom message

In [ ]:
params = {
    "phone": "+1234567890",
    "message": "Test SMS from EventReport Jupyter notebook"
}

response = requests.post(
    f"{BASE_URL}/sms/test",
    params=params,
    headers=get_headers()
)
pretty_print(response)

---
# API Summary

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| GET | `/` | Health check | No |
| POST | `/auth/register` | Register new user | No |
| POST | `/auth/login` | Login user | No |
| GET | `/auth/me` | Get current user | Yes |
| POST | `/auth/forgot-password` | Request password reset | No |
| POST | `/auth/reset-password` | Reset password | No |
| POST | `/events` | Create event | Yes |
| GET | `/events` | List events | No |
| GET | `/events/geojson` | Events as GeoJSON | No |
| GET | `/events/nearby` | Nearby events | No |
| GET | `/events/{id}` | Get event | No |
| PUT | `/events/{id}` | Update event | Yes |
| DELETE | `/events/{id}` | Delete event | Yes |
| POST | `/events/{id}/image` | Upload image | Yes |
| GET | `/events/{id}/image` | Get event image | No |
| GET | `/images/{id}` | Get image by ID | No |
| POST | `/admins` | Create admin | Yes |
| GET | `/admins` | List admins | Yes |
| DELETE | `/admins/{id}` | Delete admin | Yes |
| POST | `/sms/test` | Test SMS | Yes |
| WS | `/ws/events` | WebSocket events | No |

---
# WebSocket Example

The WebSocket endpoint `/ws/events` broadcasts new events in real-time.

In [ ]:
try:
    import websocket
    import threading
    import time

    WS_URL = "ws://backend:8000/ws/events"
    
    def on_message(ws, message):
        print(f"Received: {message}")
    
    def on_error(ws, error):
        print(f"Error: {error}")
    
    def on_close(ws, close_status_code, close_msg):
        print("WebSocket closed")
    
    def on_open(ws):
        print("WebSocket connected! Listening for new events...")
        print("(Create a new event from the web app to see it here)")
    
    print("Starting WebSocket connection...")
    print("Note: This will listen for 30 seconds then close.")
    
    ws = websocket.WebSocketApp(
        WS_URL,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
    
    thread = threading.Thread(target=ws.run_forever)
    thread.start()
    time.sleep(30)
    ws.close()
    
except ImportError:
    print("websocket-client not installed. Run: pip install websocket-client")

---
# Data Models Reference

## AlertCode Enum
```python
GREEN = "GREEN"    # Info/Low priority
YELLOW = "YELLOW"  # Caution
ORANGE = "ORANGE"  # Warning/Danger
RED = "RED"        # Urgent/Emergency
```

## Location Model
```python
{
    "type": "Point",  # GeoJSON type
    "coordinates": [longitude, latitude],  # Note: longitude first!
    "address": "optional string"
}
```

## Event Response
```python
{
    "id": "string",
    "reported_at": "datetime",
    "location": Location,
    "alert_code": AlertCode,
    "description": "string",
    "image_id": "string or null",
    "tags": ["string"],
    "reporter_id": "string",
    "created_at": "datetime"
}
```